In [2]:
# 필요한 라이브러리 전부 임포트
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
from glob import glob
%matplotlib inline

from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
# from tensorflow.keras.applications import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model, load_model, Sequential
from keras.preprocessing.image import * 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import random
import tensorflow as tf
from tensorflow import optimizers
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as L
import shutil

from keras import regularizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
# from tensorflow.keras.callbacks import EarlyStopping

In [3]:
train_df = pd.read_excel('d:/Data_Entry_2017_del.xlsx')

In [4]:
train_df = train_df.replace('NO',0).replace('YES',1)

In [5]:
# 각 Label별로 이미지를 분류
Normal = list(train_df[train_df['Finding Labels']=='No Finding']['Image Index'])
Atelectasis = list(train_df[train_df['Finding Labels'] == 'Atelectasis']['Image Index'])
Cardiomegaly = list(train_df[train_df['Finding Labels'] == 'Cardiomegaly']['Image Index'])
Effusion = list(train_df[train_df['Finding Labels'] == 'Effusion']['Image Index'])
Infiltration = list(train_df[train_df['Finding Labels'] == 'Infiltration']['Image Index'])
Mass = list(train_df[train_df['Finding Labels'] == 'Mass']['Image Index'])
Nodule = list(train_df[train_df['Finding Labels'] == 'Nodule']['Image Index'])
Pneumonia = list(train_df[train_df['Finding Labels'] == 'Pneumonia']['Image Index'])
Pneumothorax = list(train_df[train_df['Finding Labels'] == 'Pneumothorax']['Image Index'])
Consolidation = list(train_df[train_df['Finding Labels'] == 'Consolidation']['Image Index'])
Edema = list(train_df[train_df['Finding Labels'] == 'Edema']['Image Index'])
Emphysema = list(train_df[train_df['Finding Labels'] == 'Emphysema']['Image Index'])
Fibrosis = list(train_df[train_df['Finding Labels'] == 'Fibrosis']['Image Index'])
Pleural_Thickening = list(train_df[train_df['Finding Labels'] == 'Pleural Thickening']['Image Index'])
Hernia = list(train_df[train_df['Finding Labels'] == 'Hernia']['Image Index'])

In [6]:
src1 = 'd:/BigData/NIH/images_001/images' # 이미지 불러올 파일 경로
src2 = 'd:/BigData/NIH/images_002/images'
src3 = 'd:/BigData/NIH/images_003/images'
src4 = 'd:/BigData/NIH/images_004/images'
src5 = 'd:/BigData/NIH/images_005/images'
src6 = 'd:/BigData/NIH/images_006/images'
src7 = 'd:/BigData/NIH/images_007/images'
src8 = 'd:/BigData/NIH/images_008/images'
src9 = 'd:/BigData/NIH/images_009/images'
src10 = 'd:/BigData/NIH/images_010/images'
src11 = 'd:/BigData/NIH/images_011/images'
src12 = 'd:/BigData/NIH/images_012/images'

dst1 = 'd:/data1/image/train/Normal'
dst2 = 'd:/data1/image/train/Atelectasis'
dst3 = 'd:/data1/image/train/Cardiomegaly'
dst4 = 'd:/data1/image/train/Effusion'
dst5 = 'd:/data1/image/train/Infiltration'
dst6 = 'd:/data1/image/train/Mass'
dst7 = 'd:/data1/image/train/Nodule'
dst8 = 'd:/data1/image/train/Pneumonia'
dst9 = 'd:/data1/image/train/Pneumothorax'
dst10 = 'd:/data1/image/train/Consolidation'
dst11 = 'd:/data1/image/train/Edema'
dst12 = 'd:/data1/image/train/Emphysema'
dst13 = 'd:/data1/image/train/Fibrosis'
dst14 = 'd:/data1/image/train/Pleural_Thickening'
dst15 = 'd:/data1/image/train/Hernia'

In [7]:
# 분류된 이미지 리스트의 마지막에 경로를 추가
Normal.append(dst1)
Atelectasis.append(dst2)
Cardiomegaly.append(dst3)
Effusion.append(dst4)
Infiltration.append(dst5)
Mass.append(dst6)
Nodule.append(dst7)
Pneumonia.append(dst8)
Pneumothorax.append(dst9)
Consolidation.append(dst10)
Edema.append(dst11)
Emphysema.append(dst12)
Fibrosis.append(dst13)
Pleural_Thickening.append(dst14)
Hernia.append(dst15)

In [8]:
# 질병별로 분류된 각 이미지 파일이름 리스트를 하나의 리스트에 넣어준다. 
Diseases = [Normal, Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, Consolidation, Edema, Emphysema, Fibrosis, Pleural_Thickening, Hernia]
# sorce폴더의 경로를 하나의 리스트로 모아준다.
src_folders = [src1, src2, src3, src4, src5, src6, src7, src8, src9, src10, src11, src12 ]

In [9]:
src_folders

['d:/BigData/NIH/images_001/images',
 'd:/BigData/NIH/images_002/images',
 'd:/BigData/NIH/images_003/images',
 'd:/BigData/NIH/images_004/images',
 'd:/BigData/NIH/images_005/images',
 'd:/BigData/NIH/images_006/images',
 'd:/BigData/NIH/images_007/images',
 'd:/BigData/NIH/images_008/images',
 'd:/BigData/NIH/images_009/images',
 'd:/BigData/NIH/images_010/images',
 'd:/BigData/NIH/images_011/images',
 'd:/BigData/NIH/images_012/images']

In [ ]:
for src in src_folders:
    # No Finding
    for disease in Diseases:
        for file_name in disease:
            if file_name == disease[-1]:
                break
            img_src = src + '/' + file_name
            img_dst = disease[-1] + '/' + file_name
            # Create destination directory if it doesn't exist
            os.makedirs(os.path.dirname(disease[-1] + '/'), exist_ok=True)
            print(src)
            print(img_src)
            print(img_dst)
            try:
                shutil.copyfile(img_src, img_dst)
                print(f"File copied: {img_src} -> {img_dst}")
            except FileNotFoundError: 
                print('error')
                continue